In [3]:
Lines=open('../input/movie_lines.txt',encoding='utf=8',errors='ignore').read().split('\n')

In [4]:
conversation=open('../input/movie_conversations.txt',encoding='utf=8',errors='ignore').read().split('\n')

In [5]:
l=pd.DataFrame(Lines)
c=pd.DataFrame(conversation)

In [6]:
print(l.head())
print(c.head())

                                                   0
0  L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++...
1  L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON ++...
2  L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$...
3  L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++...
4  L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$...
                                                   0
0  u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L19...
1  u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']
2  u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L20...
3  u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L20...
4  u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']


In [7]:
#Lines

In [8]:
id2line={} # making dictionary for id : line data 
for line in Lines:
    _line=line.split(' +++$+++ ') #saving it in temp veriable
   # print(_line)
    if len(_line)==5:  #L925  u0  m0  BIANCA Tons
        id2line[_line[0]]=_line[4]
    
    

In [9]:
#id2line

In [10]:
#conversation


In [11]:
conversationIds=[]
for conver in conversation[:-1]:# last element is a blank ...(-1),removing [] by [1:-1],then will use replace 
    _conversation=conver.split(' +++$+++ ')[-1][1:-1].replace("'",'').replace(' ','')
    conversationIds.append(_conversation.split(','))
    

In [12]:
#conversationIds

In [13]:
Questions=[]
Answers=[]
for conver in conversationIds:
    for i in range(len(conver)-1):
        Questions.append(id2line[conver[i]])# id2line is a dictionary ,conver [i] becames the key
        Answers.append(id2line[conver[i+1]])


In [14]:
#Questions


In [15]:
#Answers

In [16]:
# now cleaning n lower the text to prepare input for CB DNLP

In [17]:
def cleaningData (text):
    text=text.lower()
    text=re.sub(r"i'm"," i am",text)
    text=re.sub(r"he's"," he is",text)
    text=re.sub(r"she's"," she is",text)
    text=re.sub(r"that's"," that is",text)
    text=re.sub(r"what's"," what is",text)
    text=re.sub(r"where's"," where is",text)
    text=re.sub(r"their's"," their is",text)
    text=re.sub(r"who's"," who is",text) 
    text=re.sub(r"\*'ll"," will",text)
    text=re.sub(r"\'ve"," have",text)
    text=re.sub(r"\'re"," are",text)
    text=re.sub(r"\'d"," would",text)
    text=re.sub(r"don't"," do not",text)
    text=re.sub(r"we'd"," we would",text)
    text=re.sub(r"won't"," will not",text)
    text=re.sub(r"can't"," can not",text)
    text=re.sub(r"[-()<>\"#@/:;{}+=*?,|]","",text)
    text=re.sub(r"it's"," it is",text)
    text=re.sub(r"you're"," you are",text)
    text=re.sub(r"doesn't"," does not",text)
    text=re.sub(r"we're"," we are",text)
    text=re.sub(r"haven't"," have not",text)
    text=re.sub(r"\*'s"," is",text)
    text=re.sub(r"aren't"," are not",text)
    text=re.sub(r"didn't "," did not",text)
    return text

    
              
        

In [18]:
#now cleanng ans que
clean_questions=[]
clean_answers=[]
for question in Questions:
    clean_questions.append(cleaningData(question))
    
for answer in Answers:
    clean_answers.append(cleaningData(answer))

In [19]:
#clean_answers

In [20]:
#clean_questions

In [21]:
# counting number of arrival of a word in que and ans
word2count={}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word]=1
        else:
            word2count[word]+=1
            
for answers in clean_answers:
    for word in answers.split():
        if word not in word2count:
            word2count[word]=1
        else:
            word2count[word]+=1
            

In [22]:
threshold=20
que_word2int={}
word_num=0
for word,count in word2count.items(): #key:value
    if count>=threshold:
        que_word2int[word]=word_num
        word_num+=1
        
ans_word2int={}
word_num=0
for word,count in word2count.items(): #key:value
    if count>=threshold:
        ans_word2int[word]=word_num
        word_num+=1
        
    

In [4]:
'''for a in zip(clean_questions,clean_answers):
    print(a)
    print('\n'*2)'''

"for a in zip(clean_questions,clean_answers):\n    print(a)\n    print('\n'*2)"

In [24]:
#que_word2int

In [25]:
tokens=['<PAD>','<EOS>','<OUT>','<SOS>']
for token in tokens:
    que_word2int[token]=len(que_word2int)+1
    
for token in tokens:
    ans_word2int[token]=len(ans_word2int)+1
    

In [5]:
#que_word2int

In [27]:
ans_int2word_ivr= {w_i:w for w,w_i in ans_word2int.items()} # inversing the dictionary

In [28]:
#ans_int2word_ivr


In [29]:
for i in range(len(clean_answers)):  # adding ending tokken for decoder 
    clean_answers[i]+='<EOS>'

In [30]:
#clean_answers

In [31]:
que2int=[] 
for question in clean_questions:
    ints=[]
    for word in question.split():
        if word not in que_word2int:
            ints.append(que_word2int['<OUT>'])
        else:
            ints.append(que_word2int[word])
    que2int.append(ints)
    
ans2int=[] 
for answer in clean_answers:
    ints=[]
    for word in answer.split():
        if word not in ans_word2int:
            ints.append(ans_word2int['<OUT>'])
        else:
            ints.append(ans_word2int[word])
    ans2int.append(ints)

In [32]:
sort_clean_que=[]
sort_clean_ans=[]
for lenghth in range(1,25+1):   # 1,26
    for i in enumerate(que2int):
        if len(i[1])==lenghth:
            sort_clean_que.append(que2int[i[0]])
            sort_clean_ans.append(ans2int[i[0]])

In [33]:
who


Answers	 Lines	 Questions	 a	 ans2int	 ans_int2word_ivr	 ans_word2int	 answer	 answers	 
c	 clean_answers	 clean_questions	 cleaningData	 conver	 conversation	 conversationIds	 count	 i	 
id2line	 ints	 l	 lenghth	 line	 np	 os	 pd	 que2int	 
que_word2int	 question	 re	 sort_clean_ans	 sort_clean_que	 tf	 threshold	 time	 token	 
tokens	 word	 word2count	 word_num	 


In [34]:
# Upto this point data preprocessing is done

# Upto this point data preprocessing is done

In [35]:
def model_inputs():
    inputs=tf.placeholder(tf.int32,[none][none],name='input')
    targets=tf.placeholder(tf.int32,[none][none],name='target')
    lr=tf.placeholder(tf.float32,name='learning_rate')
    keep_prob=tf.placeholder(tf.float32,name='keep_prob')
    return  inputs,targets,lr,keep_prob

# Now tensorflow and real chat bot development starts from here, see the 2nd part